In [3]:
# Adversarial training using the ALUM algorithm overrides the usual training step method to generate the adversarial loss alongside the normal loss. Here, we demonstrate how to perform adversarial training and then evaluate using the primitive trainer class. Then, we show how this can be integrated into an automated pipeline using Flows.

In [ ]:
# Load dummy train and test data from package
TRAIN_PATH = pkg_resources.resource_filename(
            'doggmentator', 'support/unittest-squad.json')
EVAL_PATH = pkg_resources.resource_filename(
            'doggmentator', 'support/unittest-squad.json')

In [46]:
from doggmentator.trainer.run_pipeline import _setup
hparams = {
            "model_type" : "distilbert",
            "model_name_or_path" : "distilbert-base-uncased",
            "output_dir" : "./example_outputs",
            "cache_dir" : "./example_outputs",
            "data_dir" : "./example_outputs",
            "train_file_path" : TRAIN_PATH,
            "predict_file_path" : {"dev-v1.1": EVAL_PATH},
            "aug_file_path" : None,
            "do_aug" : False,
            "do_alum" : True,
            "alpha" : 5,
            "eps" : 1e-4,
            "eta" : 1e-5,
            "sigma" : 1e-3,
            "do_train" : True,
            "do_eval" : True,
            "per_device_train_batch_size" : 2,
            "per_device_eval_batch_size" : 1,
            "gradient_accumulation_steps" : 2,
            "eval_all_checkpoints" : False,
            "num_train_epochs" : 1,
            "max_steps" : -1,
            "save_steps" : 1,
            "seed" : 512,
            "fp16" : False
}
# Initialize args
parser = HfArgumentParser(dataclass_types=[ModelArguments, TrainingArguments])
model_args, training_args = parser.parse_json_file(config_path)
model, tokenizer, train_dataset = _setup(model_args, training_args)

In [ ]:
# Construct a Flow pipeline consisting of a parent step (alum training) conditionally followed by a child step (evaluation)
f = build_flow(
            (model_args, training_args),
            model=model,
            tokenizer=tokenizer,
            train_dataset=train_dataset)
if f:
    f.run()

In [ ]:
f.result